통계적 사고 (2판) 연습문제 ([thinkstats2.com](thinkstats2.com), [think-stat.xwmooc.org](http://think-stat.xwmooc.org))<br>
Allen Downey / 이광춘(xwMOOC)

In [2]:
from __future__ import print_function, division

import first
import hypothesis
import scatter
import thinkstats2

import numpy as np

## 연습문제 9.1

표본크기가 증가함에 따라, 가설검정력은 증가하는데, 효과가 실제하면 좀더 양성임을 의미한다. 반대로, 표본크기가 줄어들면, 검정력은 설사 효과가 실제한다고 하더라도 덜 양성일 것 같다.
이런 작동방식을 조사하는데, NSFG 데이터에서 다른 일부 데이터를 갖는 검정을 실시한다. `thinkstats2.SampleRows`을 사용해서, 데이터프레임에 임의로 행일부를 선택한다.

표본크기가 감소함에 따라 검정 p-값에는 무슨 일이 일어나는가? 양의 검정을 산출하는 최소 표본크기는 얼마인가?

In [8]:
def RunTests(live, iters=1000):
    """Runs the tests from Chapter 9 with a subset of the data.

    live: DataFrame
    iters: how many iterations to run
    """
    n = len(live)
    firsts = live[live.birthord == 1]
    others = live[live.birthord != 1]

    # compare pregnancy lengths
    data = firsts.prglngth.values, others.prglngth.values
    ht = hypothesis.DiffMeansPermute(data)
    p1 = ht.PValue(iters=iters)

    data = (firsts.totalwgt_lb.dropna().values,
            others.totalwgt_lb.dropna().values)
    ht = hypothesis.DiffMeansPermute(data)
    p2 = ht.PValue(iters=iters)

    # test correlation
    live2 = live.dropna(subset=['agepreg', 'totalwgt_lb'])
    data = live2.agepreg.values, live2.totalwgt_lb.values
    ht = hypothesis.CorrelationPermute(data)
    p3 = ht.PValue(iters=iters)

    # compare pregnancy lengths (chi-squared)
    data = firsts.prglngth.values, others.prglngth.values
    ht = hypothesis.PregLengthTest(data)
    p4 = ht.PValue(iters=iters)

    print('%d\t%0.2f\t%0.2f\t%0.2f\t%0.2f' % (n, p1, p2, p3, p4))
    
thinkstats2.RandomSeed(18)

live, firsts, others = first.MakeFrames()

n = len(live)
for _ in range(7):
    sample = thinkstats2.SampleRows(live, n)
    RunTests(sample)
    n //= 2

9148	0.16	0.00	0.00	0.00
4574	0.10	0.01	0.00	0.00
2287	0.25	0.06	0.00	0.00
1143	0.24	0.03	0.39	0.03
571	0.81	0.00	0.04	0.04
285	0.57	0.41	0.48	0.83
142	0.45	0.08	0.60	0.04


test1: 평균 임신기간 차이
test2: 평균 출생체중 차이
test3: 산모 연령과 출생체중 상관관계
test4: 임신기간 카이제곱 검정

n       test1   test2   test2   test4  
9148 &rarr;	0.16 &rarr;	0.00 &rarr;	0.00 &rarr;	0.00  
4574&rarr;	0.10&rarr;	0.01&rarr;	0.00&rarr;	0.00  
2287&rarr;	0.25&rarr;	0.06&rarr;	0.00&rarr;	0.00  
1143&rarr;	0.24&rarr;	0.03&rarr;	0.39&rarr;	0.03  
571&rarr;	0.81&rarr;	0.00&rarr;	0.04&rarr;	0.04  
285&rarr;	0.57&rarr;	0.41&rarr;	0.48&rarr;	0.83  
142&rarr;	0.45&rarr;	0.08&rarr;	0.60&rarr;	0.04

결론: 예상했듯이, 데이터를 제거함에 따라 더큰 표본크기를 갖는 양의 검정은 음이 된다. 하지만, 작은 표본크기에서 조차도 일부 양의 값을 갖는 등 패턴은 일정치 않다.

## 연습문제 9.2  
9.3절처럼, 순열로 귀무가설을 모의시험했다; 즉, 관측된 값을 마치 전체 모집단을 대표하는 것처럼 다루었고, 무작위로 모집단의 구성원을 두 집단에 배정했다.
대안은 표본을 사용해서 모집단 분포를 추정하고 나서, 분포로부터 임의 표본을 추출하는 것이다. 이런 과정을 재표집(resampling)이라고 부른다. 재표집을 구현하는 몇가지 방식이 있지만, 가장 단순한 것중 하나가 9.10 처럼 관측된 값에서 복원방식으로 표본을 추출하는 것이다.

DiffMeansPermute에서 상속받고, 순열보다 재표집을 구현하는 RunModel을 치환(override)하는 클래스 DiffMeansResample을 작성하시오.

이 모형을 사용해서 임신기간과 출생체중 사이 차이를 검정하시오. 이 모형이 결과에 얼마나 영향을 주는가?

In [9]:
class DiffMeansResample(hypothesis.DiffMeansPermute):
    """Tests a difference in means using resampling."""
    
    def RunModel(self):
        """Run the model of the null hypothesis.

        returns: simulated data
        """
        group1 = np.random.choice(self.pool, self.n, replace=True)
        group2 = np.random.choice(self.pool, self.m, replace=True)
        return group1, group2
  

def RunResampleTest(firsts, others):
    """Tests differences in means by resampling.

    firsts: DataFrame
    others: DataFrame
    """
    data = firsts.prglngth.values, others.prglngth.values
    ht = DiffMeansResample(data)
    p_value = ht.PValue(iters=10000)
    print('\nmeans permute preglength')
    print('p-value =', p_value)
    print('actual =', ht.actual)
    print('ts max =', ht.MaxTestStat())

    data = (firsts.totalwgt_lb.dropna().values,
            others.totalwgt_lb.dropna().values)
    ht = hypothesis.DiffMeansPermute(data)
    p_value = ht.PValue(iters=10000)
    print('\nmeans permute birthweight')
    print('p-value =', p_value)
    print('actual =', ht.actual)
    print('ts max =', ht.MaxTestStat())

RunResampleTest(firsts, others)        


means permute preglength
p-value = 0.1675
actual = 0.0780372667775
ts max = 0.225502935911

means permute birthweight
p-value = 0.0
actual = 0.124761184535
ts max = 0.116868397475


순열대신에 재표추출을 사용하는 것은 결과에 거의 영향이 없다.

두 모형은 약간 다른 가정에 기반하고 있고, 상기 예제에서 이것 혹은 저것을 선택할 강력한 사유는 없다.  하지만, 일반적으로 p-값은 귀무가설 선택에 달려있다; 서로 다른 모형은 매우 다른 결과를 이끌어낼 수 있다.